# AGI Brain Simulation

**Spiking Neural Network Cognitive Architecture**

This notebook runs the AGI brain simulation with GPU acceleration on Google Colab.

## Features
- 125K+ neuron reasoning architecture
- Savant-mode memory (perfect retention)
- Neural plasticity and learning (STDP)
- Embodied agent with navigation
- GPU-accelerated simulation

---

**Runtime Setup:** Go to `Runtime > Change runtime type > T4 GPU`

## 1. Setup Environment

In [ ]:
# Install dependencies
!pip install nengo nengo-spa tensorflow matplotlib numpy --quiet

# Check GPU
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
gpus = tf.config.list_physical_devices('GPU')
print("GPUs available:", gpus)
if gpus:
    print("GPU Name:", tf.test.gpu_device_name())

In [ ]:
# Clone the repository
!rm -rf agi-brain  # Remove if exists
!git clone https://github.com/jeebus87/agi-brain.git
%cd agi-brain

In [ ]:
# Add to path
import sys
sys.path.insert(0, '.')

# Imports
import numpy as np
import matplotlib.pyplot as plt
import nengo
import nengo_spa as spa

print("Nengo version:", nengo.__version__)
print("Setup complete!")

## 2. Reasoning Demonstration

The 100K+ neuron reasoning architecture with:
- Working memory (savant mode)
- Rule application engine
- Analogy engine
- Executive controller

In [ ]:
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='nengo_spa')

from src.reasoning.poc_100k import ReasoningPOC100K, count_neurons

# Create the 100K neuron POC
print("Building 100K neuron reasoning architecture...")
poc = ReasoningPOC100K(dimensions=64)

# Count neurons
counts = count_neurons(poc)
print(f"\nTotal neurons: {counts['total']:,}")
print("\nComponents:")
for name, count in counts.items():
    if name != 'total':
        print(f"  {name}: {count:,}")

In [ ]:
# Run a reasoning simulation
print("Running reasoning simulation (5 seconds)...")

with nengo.Simulator(poc, progress_bar=True) as sim:
    sim.run(5.0)

print("Simulation complete!")

In [ ]:
# Visualize reasoning activity
t = sim.trange()

fig, axes = plt.subplots(3, 1, figsize=(14, 10))

# Working memory
ax1 = axes[0]
wm_data = sim.data[poc.p_wm]
im1 = ax1.imshow(wm_data[:, :16].T, aspect='auto', cmap='RdBu', extent=[0, t[-1], 16, 0])
ax1.set_ylabel('Dimension')
ax1.set_title('Working Memory State (Savant Mode)', fontweight='bold')
plt.colorbar(im1, ax=ax1)

# Rule application
ax2 = axes[1]
rule_data = sim.data[poc.p_rule_conclusion]
im2 = ax2.imshow(rule_data[:, :16].T, aspect='auto', cmap='viridis', extent=[0, t[-1], 16, 0])
ax2.set_ylabel('Dimension')
ax2.set_title('Rule Application Output', fontweight='bold')
plt.colorbar(im2, ax=ax2)

# Response output
ax3 = axes[2]
response_data = sim.data[poc.p_response]
ax3.plot(t, response_data[:, :4], alpha=0.8)
ax3.set_xlabel('Time (seconds)')
ax3.set_ylabel('Activity')
ax3.set_title('Response Generator Output', fontweight='bold')
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Learning Demonstration

Neural plasticity using PES learning rule - the network learns to produce a specific output.

In [ ]:
# Simple learning demo
print("Creating learning network...")
print("Task: Learn to output 0.8 when input is 1.0")

with nengo.Network(seed=42) as learning_model:
    # Constant input of 1.0
    stim = nengo.Node(output=1.0)
    
    # Pre-synaptic ensemble
    pre = nengo.Ensemble(100, dimensions=1)
    nengo.Connection(stim, pre)
    
    # Post-synaptic ensemble
    post = nengo.Ensemble(100, dimensions=1)
    
    # Learnable connection - starts at 0
    conn = nengo.Connection(
        pre, post,
        transform=0,
        learning_rule_type=nengo.PES(learning_rate=3e-4)
    )
    
    # Error signal: target (0.8) minus actual
    error = nengo.Node(output=lambda t, x: x - 0.8, size_in=1)
    nengo.Connection(post, error)
    nengo.Connection(error, conn.learning_rule)
    
    # Probes
    p_post = nengo.Probe(post, synapse=0.01)
    p_error = nengo.Probe(error, synapse=0.01)

print("Running learning simulation (10 seconds)...")
with nengo.Simulator(learning_model, progress_bar=True) as sim:
    sim.run(10.0)

print("\nResults:")
early_output = sim.data[p_post][500:1000].mean()
late_output = sim.data[p_post][-500:].mean()
print(f"  Target: 0.80")
print(f"  Early output (t=0.5-1s): {early_output:.3f}")
print(f"  Final output (t=9.5-10s): {late_output:.3f}")
print(f"  Status: {'SUCCESS!' if abs(late_output - 0.8) < 0.15 else 'Learning...'}")

In [ ]:
# Visualize learning
t = sim.trange()

fig, axes = plt.subplots(2, 1, figsize=(12, 6))

ax1 = axes[0]
ax1.plot(t, sim.data[p_post], label='Learned Output', linewidth=2)
ax1.axhline(y=0.8, color='red', linestyle='--', label='Target (0.8)', linewidth=2)
ax1.fill_between([0, 1], -0.5, 1.5, alpha=0.2, color='blue', label='Early')
ax1.fill_between([9, 10], -0.5, 1.5, alpha=0.2, color='green', label='Late')
ax1.set_ylabel('Output')
ax1.set_title('Learning Progress: Output Approaches Target', fontweight='bold')
ax1.legend(loc='right')
ax1.set_ylim(-0.5, 1.5)
ax1.grid(True, alpha=0.3)

ax2 = axes[1]
ax2.plot(t, sim.data[p_error], color='red', alpha=0.7)
ax2.axhline(y=0, color='gray', linestyle='--')
ax2.set_xlabel('Time (seconds)')
ax2.set_ylabel('Error')
ax2.set_title('Error Signal (approaches 0 as learning succeeds)', fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Embodied Navigation

Neural agent navigating a GridWorld environment.

In [ ]:
from src.integration.embodiment.environment import GridWorld
from src.integration.embodiment.agent import EmbodiedAgent, AgentConfig

# Create environment
env = GridWorld(
    size=8,
    n_goals=1,
    n_obstacles=5,
    n_hazards=2,
    vision_range=3,
    max_steps=50,
    seed=42
)

print("GridWorld Environment:")
print(env.render_ascii())
print("\nLegend: A=Agent, G=Goal, #=Wall, X=Hazard, .=Empty")

In [ ]:
# Create embodied agent
config = AgentConfig(
    vocab_dimensions=64,
    n_neurons_per_dim=30,
    seed=42
)

agent = EmbodiedAgent(env, config)
print(f"Embodied Agent: {agent.get_neuron_count():,} neurons")

# Build simulator
agent.build_simulator(progress_bar=False)
print("Agent ready!")

In [ ]:
# Run episodes
print("Running navigation episodes...\n")

results = []
for ep in range(5):
    env.rng = np.random.default_rng(42 + ep)
    env._generate_grid()
    
    stats = agent.run_episode(max_steps=50, sim_time_per_step=0.05)
    results.append(stats)
    
    status = "SUCCESS" if stats['success'] else "timeout"
    print(f"Episode {ep+1}: reward={stats['total_reward']:6.2f}, steps={stats['steps']:2d}, {status}")

print(f"\nSuccess rate: {np.mean([r['success'] for r in results])*100:.0f}%")

agent.close()

## 5. GPU Acceleration Benchmark

Compare CPU vs GPU performance for neural simulations.

In [ ]:
from src.acceleration.gpu_backend import GPUAccelerator, NeuronPopulationGPU, check_gpu_available
from src.acceleration.benchmarks import run_benchmark, compare_backends

# Check GPU status
info = check_gpu_available()
print("GPU Status:")
print(f"  TensorFlow: {info['tensorflow_available']}")
print(f"  GPU Available: {info['gpu_available']}")
print(f"  Devices: {info['gpu_devices']}")
print(f"  Recommended: {info['recommended_backend']}")

In [ ]:
# Run benchmarks
print("\nRunning performance benchmarks...\n")

neuron_counts = [1000, 5000, 10000, 25000, 50000]

results = compare_backends(
    neuron_counts=neuron_counts,
    dimensions=64,
    sim_time=0.5,
    backends=['nengo', 'gpu']
)

In [ ]:
# Visualize benchmark results
fig, ax = plt.subplots(figsize=(10, 6))

for backend, backend_results in results.items():
    valid = [r for r in backend_results if r]
    if valid:
        neurons = [r.n_neurons for r in valid]
        rates = [r.neurons_per_second / 1e6 for r in valid]
        ax.plot(neurons, rates, 'o-', label=backend, linewidth=2, markersize=8)

ax.set_xlabel('Number of Neurons')
ax.set_ylabel('Million Neurons/Second')
ax.set_title('Performance Comparison: CPU vs GPU Backend', fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_xscale('log')

plt.tight_layout()
plt.show()

# Summary
print("\nBest throughput achieved:")
for backend, backend_results in results.items():
    valid = [r for r in backend_results if r]
    if valid:
        best = max(valid, key=lambda r: r.neurons_per_second)
        print(f"  {backend}: {best.neurons_per_second/1e6:.2f}M neurons/sec @ {best.n_neurons:,} neurons")

## 6. Large-Scale Simulation

Simulate 100K+ neurons with GPU acceleration.

In [ ]:
from src.acceleration.gpu_backend import LargeScaleSimulator
import time

# Create large-scale simulator
print("Creating large-scale simulator...")
large_sim = LargeScaleSimulator(
    n_populations=10,
    neurons_per_population=10000,  # 100K total
    dimensions=64,
    device='auto'
)

# Create input signal
t = np.linspace(0, 1, 1000)
input_signal = np.sin(2 * np.pi * t)[:, np.newaxis] * np.ones(64)
input_signal = input_signal.astype(np.float32)

# Run simulation
print(f"\nSimulating {large_sim.total_neurons:,} neurons for 1 second...")
start = time.perf_counter()
activities = large_sim.run(input_signal, dt=0.001)
elapsed = time.perf_counter() - start

print(f"\nResults:")
print(f"  Wall time: {elapsed:.2f} seconds")
print(f"  Speedup: {1.0/elapsed:.2f}x real-time")
print(f"  Throughput: {large_sim.total_neurons * 1000 / elapsed / 1e6:.2f}M neuron-timesteps/sec")

In [ ]:
# Visualize large-scale activity
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Population activity heatmap
ax1 = axes[0]
pop0 = activities['pop_0']
im = ax1.imshow(pop0[:, :100].T, aspect='auto', cmap='hot', extent=[0, 1, 100, 0])
ax1.set_xlabel('Time (seconds)')
ax1.set_ylabel('Neuron Index')
ax1.set_title(f'Population Activity ({large_sim.total_neurons:,} total neurons)', fontweight='bold')
plt.colorbar(im, ax=ax1, label='Firing Rate')

# Mean activity per population
ax2 = axes[1]
for i in range(min(5, len(activities))):
    mean_act = np.mean(activities[f'pop_{i}'], axis=1)
    ax2.plot(t, mean_act, label=f'Pop {i}', alpha=0.8)

ax2.set_xlabel('Time (seconds)')
ax2.set_ylabel('Mean Activity')
ax2.set_title('Population Averages Over Time', fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## Summary

This notebook demonstrated:

1. **Reasoning Architecture** - 125K+ neurons with working memory, rule application, and executive control
2. **Neural Learning** - PES-based plasticity that learns input-output mappings
3. **Embodied Navigation** - Agent navigating GridWorld using neural perception and action
4. **GPU Acceleration** - TensorFlow backend for high-performance simulation
5. **Large-Scale Simulation** - 100K neurons running efficiently

### Next Steps
- Scale to 1M+ neurons with distributed simulation
- Add more sophisticated learning (reward-modulated STDP)
- Connect to richer environments (MuJoCo, Unity)
- Integrate LLM for language capabilities